In [1]:
import cv2
import numpy as np
from pygame import mixer
from tensorflow.keras.models import load_model
from pyfirmata import Arduino, util
import time
import serial 

pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

# Initialize pygame mixer
mixer.init()
sound = mixer.Sound(r'D:\Davies Ningthoujam\My project\Ml Project\Driver drowsiness Detection\alarm.wav')

# Load the cascades and model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
model = load_model(r'D:\Davies Ningthoujam\My project\Ml Project\Driver drowsiness Detection\models')


In [3]:
# Connect to Arduino (for local PWM control)
board = Arduino('COM3')  # Replace 'COM3' with your Arduino's port if necessary
#motor_pwm_pin = board.get_pin('d:3:p')  # Define a PWM pin for the motor (D3 in this example)


In [4]:
# Connect to ESP8266 via Serial
#esp8266_port = 'COM3'  # Replace with your ESP8266's port
#baud_rate = 9600
#ser = serial.Serial(esp8266_port, baud_rate)
#time.sleep(2)  # Wait for the ESP8266 to initialize

Score = 0  # Initial drowsiness score



In [5]:
def send_command(value):
    """Sends a command to the Arduino to control the motor."""
    if value == '1':
        motor_pwm_pin.write(0.5)  # Apply brake (slow down motor)
        print("Motor slowed down")
    elif value == '0':
        motor_pwm_pin.write(1.0)  # Release brake (restore full speed)
        print("Motor full speed restored")
    else:
        print("Invalid value. Only '1' or '0' allowed.")

In [6]:
# Start video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera not found!")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to grab frame.")
        break
    
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    
    # Adjust brightness based on lighting conditions
    brightness = np.mean(gray)
    if brightness > 180:
        frame = cv2.convertScaleAbs(frame, alpha=0.5, beta=0)
    elif brightness < 50:
        frame = cv2.convertScaleAbs(frame, alpha=1.5, beta=20)

    # Detect faces and eyes in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    if len(faces) == 0:
        print("No faces detected.")
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 3)
        face_roi = gray[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.2, minNeighbors=3)
        if len(eyes) == 0:
            print("No eyes detected.")
        for (ex, ey, ew, eh) in eyes:
            ex, ey = x + ex, y + ey  # Adjust coordinates
            eye = frame[ey:ey + eh, ex:ex + ew]
            eye = cv2.resize(eye, (80, 80)) / 255.0
            eye = np.expand_dims(eye.reshape(80, 80, 3), axis=0)
            prediction = model.predict(eye)
            print(f"Prediction: {prediction}")
            if prediction[0][0] > 0.30:  # Closed eye detected
                Score += 1
                if Score > 20:
                    sound.play()
                    send_command('1')  # Apply brake (slow down motor)
                     
            elif prediction[0][1] > 0.65:  # Open eye detected
                Score = max(Score - 2, 0)  # Decrease score, but keep it above 0
                if Score < 10:
                    send_command('0')  # Release brake (restore full speed)

    # Display real-time score and alerts on frame
    cv2.putText(frame, f'Score: {Score}', (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow('Driver Drowsiness Detection', frame)

    # Quit if 'q' is pressed
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

    cap.release()
    cv2.destroyAllWindows()
    #ser.close()
    board.exit()


1/1 [==============================] - 3s 3s/step
Prediction: [[0.0044624  0.99553764]]


NameError: name 'motor_pwm_pin' is not defined